In [13]:
import torch
from torch.utils import data
from torch import nn
import numpy as np
import math
from d2l import torch as d2l

$f(x)=5+1.2x-3.4\frac{x^2}{2!}+5.6\frac{x^3}{3!}+\epsilon\ where\ \epsilon\sim N(0, 0.01)$

In [6]:
max_degree = 20
n_train, n_test = 100, 100
w_true = np.zeros(max_degree)
w_true[0:4] = [5, 1.2, -3.4, 5.6]
print(w_true.shape) # [20]

features = np.random.normal(size = (n_train + n_test, 1))
print(features.shape)
np.random.shuffle(features) # [200, 1]
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1)) # [1, 20]
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)
    
print(poly_features.shape) # [200, 20]
labels = np.dot(poly_features, w_true)
print(labels.shape)
labels = np.random.normal(scale = 0.1, size = labels.shape)

(20,)
(200, 1)
(200, 20)
(200,)


In [8]:
x = np.array([1, 2, 3, 4])
y = np.array([[1, 1, 1, 1], [1, 1, 1, 1]])
print(x.shape, y.shape)
np.dot(y, x)

(4,) (2, 4)


array([10, 10])

In [12]:
w_true, features, poly_features, labels = [
                                               torch.tensor(i, dtype=torch.float32) 
                                               for i in [w_true, features, poly_features, labels]
                                           ]

/home/shao/anaconda3/envs/ai/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
def evaluate_loss(net, data_iter, loss):
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        y_hat = net(X)
        y = y.reshpe(y_hat.shape)
        l = loss(y_hat, y)
        metric.add(l.sum(), l.numel())
        
    return metric[0] / metric[1]

In [ ]:
def train(train_features, test_features, train_label, test_labels):
    input_shape = train_features.shape[-1]
    
    loss = nn.MSELoss()
    net = nn.Linear(input_shape, 1, bias=False)
    